# Дообучение модели на собственных данных для задачи генерации текста на
русском языке

Ноутбук подходит для запуска на бесплатном тарифе в Google Colab. Перед
запуском ячеек вы должны располагать набором данных (`corpus.db` или
`corpus_small.db`. Данные можно получить, запустив `grabber.py`.

Доустанавливаем нужные библиотеки

In [ ]:
!pip install transformers datasets evaluate

Необходимые импорты

In [ ]:
import math

from datasets import Dataset

from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from huggingface_hub import notebook_login

Необходимо указать токен 🤗, чтобы загрузить обученную модель.

In [ ]:
notebook_login()

Пожалуй, самую большую известность получили модели для русского языка от
Сбера. Её малый вариант и будем дотренировывать - модели большего размера
уже не помещаются в Colab.

In [ ]:
model_checkpoint = "sberbank-ai/rugpt3small_based_on_gpt2"

Готовим данные

In [ ]:
data = Dataset.from_sql("SELECT text FROM quotes;",
                        con="sqlite:///corpus_small.db")
data = data.train_test_split(test_size=0.25)

Перед каждой цитатой добавим затравку: "Преподаватель говорит: ". Это
позволит нам генерировать случайные цитаты без привязки к конкретному
инпуту, а также разграничивать сэмплы.

In [ ]:
data = data.map(lambda x: {"text":
                               f"Преподаватель  говорит: "
                               f"{x['text']}<|endoftext|>"})

Токенизируем и разбиваем на блоки

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples['text'])

In [ ]:
data = data.map(preprocess_function, batched=True, remove_columns=["text"])

В идеале, текст надо делить на блоки, длина которых соответствует длине
блоков, на которых была обучена исходная модель. Однако слишком длинные
блоки переполняют оперативную память. В бесплатном Colab не помещаются блоки
 длиннее 512 токенов.

In [ ]:
block_size = 512

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
data = data.map(group_texts, batched=True)
print(data)

Загружаем исходную модель

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

Задаём имя нашей новой модели и гиперпараметры обучения

In [ ]:
model_name = f"{model_checkpoint.split('/')[-1]}-finetuned_teachers_quotes"

training_args = TrainingArguments(
    model_name,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True
)

In [ ]:
new_model = Trainer(
    model=model,
    args=training_args,
    train_dataset=data["train"],
    eval_dataset=data["test"]
)

Ура, тренируем!

In [ ]:
new_model.train()

Посмотрим на циферки

In [ ]:
eval_results = new_model.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity в районе 18 - очень неплохой результат, учитывая неаккуратность
наших данных и сравнительно небольшой размер.

Выгружаем обученную модель и токенизатор на 🤗. Теперь её можно будет
использовать через API, не загружая локально (там больше 500 МБ).

In [ ]:
new_model.push_to_hub()
tokenizer.push_to_hub(f"Futyn-Maker/{model_name}")
